In [4]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set(rc={'figure.figsize':(20,8)})

# Read data

In [2]:
sale_train = pd.read_csv("sales_train_v2.csv")
test = pd.read_csv("test.csv")
item_categories = pd.read_csv("item_categories.csv")
items = pd.read_csv("items.csv")
shops = pd.read_csv("shops.csv")
sale_train['date'] = pd.to_datetime(sale_train['date'], format='%d.%m.%Y')

In [35]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [34]:
sale_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [3]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


# Data Preprocessing

In [3]:
data = sale_train.groupby(['item_id', 'shop_id', 'date_block_num']).agg({
    "item_price": "mean",
    "item_cnt_day": "sum"     
}).reset_index()

In [16]:
data_train = data.query("date_block_num < 33")
data_val = data.query("date_block_num == 33")

## Process test data

### Change columns order

In [17]:
data_test = test.drop(['ID'], axis=1)
data_test = data_test[["item_id", 'shop_id']]

In [18]:
data_test.head()

,item_id,shop_id
0,5037,5
1,5320,5
2,5233,5
3,5232,5
4,5268,5


### Add date_block_num 34 to data_test

In [19]:
data_test['date_block_num'] = 34

### Add price to data_test

In [20]:
test_price = data.groupby(['item_id', 'shop_id'])['item_price'].last().reset_index()

In [21]:
data_test = pd.merge(data_test, test_price, left_on=['shop_id', 'item_id'], right_on=['shop_id', 'item_id'], 
                     how='left') 

In [22]:
data_test.head()

,item_id,shop_id,date_block_num,item_price
0,5037,5,34,749.5
1,5320,5,34,NaN
2,5233,5,34,1199.0
3,5232,5,34,599.0
4,5268,5,34,NaN


In [23]:
data_test.shape

(214200, 4)

In [24]:
data_test.isna().sum()

item_id                0
shop_id                0
date_block_num         0
item_price        102796
dtype: int64

### Merge data_test with train_data for easier features generation

In [25]:
data_test['item_cnt_day'] = -9999

In [26]:
data_test.head()

,item_id,shop_id,date_block_num,item_price,item_cnt_day
0,5037,5,34,749.5,-9999
1,5320,5,34,NaN,-9999
2,5233,5,34,1199.0,-9999
3,5232,5,34,599.0,-9999
4,5268,5,34,NaN,-9999


In [55]:
data = pd.concat([data_train, data_test])

In [56]:
print(data_train.shape)
print(data_test.shape)
print(data.shape)

(1577593, 5)
(214200, 5)
(1791793, 5)


In [57]:
data.head()

,item_id,shop_id,date_block_num,item_price,item_cnt_day
0,0,54,20,58.0,1.0
1,1,55,15,4490.0,2.0
2,1,55,18,4490.0,1.0
3,1,55,19,4490.0,1.0
4,1,55,20,4490.0,1.0


## EDA

In [5]:
data = pd.read_pickle('data.pkl')

In [6]:
data.head()

,date_block_num,item_cnt_day,item_id,item_price,shop_id,item_category_id
0,20,1.0,0,58.0,54,40
1,15,2.0,1,4490.0,55,76
2,18,1.0,1,4490.0,55,76
3,19,1.0,1,4490.0,55,76
4,20,1.0,1,4490.0,55,76


In [16]:
data.isna().sum()

date_block_num      0
item_cnt_day        0
item_id             0
item_price          0
shop_id             0
item_category_id    0
dtype: int64

In [13]:
data.shape

(1823324, 6)

In [14]:
data.fillna(0, inplace=True)